# Task 1: Create a Prescription Parser using CRF
This task tests your ability to build a Doctor Prescription Parser with the help of CRF model

Your job is to build a Prescription Parser that takes a prescription (sentence) as an input and find / label the words in that sentence with one of the already pre-defined labels

### Problem: SEQUENCE PREDICTION - Label words in a sentence
#### Input : Doctor Prescription in the form of a sentence split into tokens
- Ex: Take 2 tablets once a day for 10 days

#### Output : FHIR Labels
- ('Take', 'Method')
- ('2', 'Qty') 
- ('tablets', 'Form')
- ('once', 'Frequency')
- ('a', 'Period') 
- ('day', 'PeriodUnit')
- ('for', 'FOR')
- ('10', 'Duration')
- ('days', 'DurationUnit') 

### Major Steps
- Install necessary library
- Import the libraries
- Create training data with labels
    - Split the sentence into tokens
    - Compute POS tags
    - Create triples
- Extract features
- Split the data into training and testing set
- Create CRF model
- Save the CRF model
- Load the CRF model
- Predict on test data
- Accuracy

#### Install necesaary library

In [179]:
import nltk
nltk.download('averaged_perceptron_tagger')

#### Import the necessary libraries

In [180]:
import joblib
from nltk.tag import pos_tag
from sklearn_crfsuite import CRF, metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

### Input data (GIVEN)
#### Creating the inputs to the ML model in the following form:
- sigs --> ['take 3 tabs for 10 days']       INPUT SIG
- input_sigs --> [['take', '3', 'tabs', 'for', '10', 'days']]      TOKENS
- output_labels --> [['Method','Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]       LABELS

In [181]:
sigs = ["for 5 to 6 days", "inject 2 units", "x 2 weeks", "x 3 days", "every day", "every 2 weeks", "every 3 days", "every 1 to 2 months", "every 2 to 6 weeks", "every 4 to 6 days", "take two to four tabs", "take 2 to 4 tabs", "take 3 tabs orally bid for 10 days at bedtime", "swallow three capsules tid orally", "take 2 capsules po every 6 hours", "take 2 tabs po for 10 days", "take 100 caps by mouth tid for 10 weeks", "take 2 tabs after an hour", "2 tabs every 4-6 hours", "every 4 to 6 hours", "q46h", "q4-6h", "2 hours before breakfast", "before 30 mins at bedtime", "30 mins before bed", "and 100 tabs twice a month", "100 tabs twice a month", "100 tabs once a month", "100 tabs thrice a month", "3 tabs daily for 3 days then 1 tab per day at bed", "30 tabs 10 days tid", "take 30 tabs for 10 days three times a day", "qid q6h", "bid", "qid", "30 tabs before dinner and bedtime", "30 tabs before dinner & bedtime", "take 3 tabs at bedtime", "30 tabs thrice daily for 10 days ", "30 tabs for 10 days three times a day", "Take 2 tablets a day", "qid for 10 days", "every day", "take 2 caps at bedtime", "apply 3 drops before bedtime", "take three capsules daily", "swallow 3 pills once a day", "swallow three pills thrice a day", "apply daily", "apply three drops before bedtime", "every 6 hours", "before food", "after food", "for 20 days", "for twenty days", "with meals"]
input_sigs = [['for', '5', 'to', '6', 'days'], ['inject', '2', 'units'], ['x', '2', 'weeks'], ['x', '3', 'days'], ['every', 'day'], ['every', '2', 'weeks'], ['every', '3', 'days'], ['every', '1', 'to', '2', 'months'], ['every', '2', 'to', '6', 'weeks'], ['every', '4', 'to', '6', 'days'], ['take', 'two', 'to', 'four', 'tabs'], ['take', '2', 'to', '4', 'tabs'], ['take', '3', 'tabs', 'orally', 'bid', 'for', '10', 'days', 'at', 'bedtime'], ['swallow', 'three', 'capsules', 'tid', 'orally'], ['take', '2', 'capsules', 'po', 'every', '6', 'hours'], ['take', '2', 'tabs', 'po', 'for', '10', 'days'], ['take', '100', 'caps', 'by', 'mouth', 'tid', 'for', '10', 'weeks'], ['take', '2', 'tabs', 'after', 'an', 'hour'], ['2', 'tabs', 'every', '4-6', 'hours'], ['every', '4', 'to', '6', 'hours'], ['q46h'], ['q4-6h'], ['2', 'hours', 'before', 'breakfast'], ['before', '30', 'mins', 'at', 'bedtime'], ['30', 'mins', 'before', 'bed'], ['and', '100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'once', 'a', 'month'], ['100', 'tabs', 'thrice', 'a', 'month'], ['3', 'tabs', 'daily', 'for', '3', 'days', 'then', '1', 'tab', 'per', 'day', 'at', 'bed'], ['30', 'tabs', '10', 'days', 'tid'], ['take', '30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['qid', 'q6h'], ['bid'], ['qid'], ['30', 'tabs', 'before', 'dinner', 'and', 'bedtime'], ['30', 'tabs', 'before', 'dinner', '&', 'bedtime'], ['take', '3', 'tabs', 'at', 'bedtime'], ['30', 'tabs', 'thrice', 'daily', 'for', '10', 'days'], ['30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['take', '2', 'tablets', 'a', 'day'], ['qid', 'for', '10', 'days'], ['every', 'day'], ['take', '2', 'caps', 'at', 'bedtime'], ['apply', '3', 'drops', 'before', 'bedtime'], ['take', 'three', 'capsules', 'daily'], ['swallow', '3', 'pills', 'once', 'a', 'day'], ['swallow', 'three', 'pills', 'thrice', 'a', 'day'], ['apply', 'daily'], ['apply', 'three', 'drops', 'before', 'bedtime'], ['every', '6', 'hours'], ['before', 'food'], ['after', 'food'], ['for', '20', 'days'], ['for', 'twenty', 'days'], ['with', 'meals']]
output_labels = [['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'], ['Method', 'Qty', 'Form'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['EVERY', 'Period'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'TID', 'PO'], ['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'BY', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'AFTER', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Q46H'], ['Q4-6H'], ['Qty', 'PeriodUnit', 'BEFORE', 'WHEN'], ['BEFORE', 'Qty', 'M', 'AT', 'WHEN'], ['Qty', 'M', 'BEFORE', 'WHEN'], ['AND', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'THEN', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN'], ['Qty', 'Form', 'Duration', 'DurationUnit', 'TID'], ['Method', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Qty', 'TIMES', 'Period', 'PeriodUnit'], ['QID', 'Q6H'], ['BID'], ['QID'],['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Duration', 'DurationUnit'], ['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Frequency', 'TIMES', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Period', 'PeriodUnit'], ['QID', 'FOR', 'Duration', 'DurationUnit'], ['EVERY', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['Method', 'Qty', 'Form', 'DAILY'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'DAILY'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['EVERY', 'Period', 'PeriodUnit'], ['BEFORE', 'FOOD'], ['AFTER', 'FOOD'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['WITH', 'FOOD']]

In [182]:
len(sigs), len(input_sigs) , len(output_labels)

(56, 56, 56)

### Creating a Tuples Maker method
Create the tuples as given below by writing a function **tuples_maker(input_sigs, output_labels)** and returns **output** as given below

Input(s): 
- input_sigs
- output_lables

Output:

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')], [second sentence], ...]

In [183]:
def tuples_maker(inp, out):
    sample_data = []
    for ip, op in zip(input_sigs, output_labels):
        temp = []
        for i, j in zip(ip, op):
            temp.append((i, j))
        sample_data.append(temp)
    return sample_data

In [184]:
whole_data = tuples_maker(input_sigs, output_labels)
whole_data

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')],
 [('inject', 'Method'), ('2', 'Qty'), ('units', 'Form')],
 [('x', 'FOR'), ('2', 'Duration'), ('weeks', 'DurationUnit')],
 [('x', 'FOR'), ('3', 'Duration'), ('days', 'DurationUnit')],
 [('every', 'EVERY'), ('day', 'Period')],
 [('every', 'EVERY'), ('2', 'Period'), ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'), ('3', 'Period'), ('days', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('1', 'Period'),
  ('to', 'TO'),
  ('2', 'PeriodMax'),
  ('months', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('2', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('4', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('days', 'PeriodUnit')],
 [('take', 'Method'),
  ('two', 'Qty'),
  ('to', 'TO'),
  ('four', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('2', 'Qty'),
  ('to', 'TO'),
  ('4', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('3', 

### Creating the triples_maker( ) for feature extraction
- input: tuples_maker_output
- output: 
[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')], [second sentence], ... ]

In [185]:
def triples_maker(whole_data):
    sample_data = []
    for sentence, input in zip(whole_data, input_sigs):
        tag = pos_tag(input)
        temp = []
        for tuple, pos in zip(sentence, tag):
            temp.append((pos[0], pos[1], tuple[1]))
        sample_data.append(temp)
    return sample_data

In [186]:
sample_data = triples_maker(whole_data)
sample_data

[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')],
 [('inject', 'JJ', 'Method'), ('2', 'CD', 'Qty'), ('units', 'NNS', 'Form')],
 [('x', 'RB', 'FOR'),
  ('2', 'CD', 'Duration'),
  ('weeks', 'NNS', 'DurationUnit')],
 [('x', 'RB', 'FOR'),
  ('3', 'CD', 'Duration'),
  ('days', 'NNS', 'DurationUnit')],
 [('every', 'DT', 'EVERY'), ('day', 'NN', 'Period')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('3', 'CD', 'Period'),
  ('days', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('1', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('2', 'CD', 'PeriodMax'),
  ('months', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('4', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('

### Creating the features extractor method (GIVEN as a BASELINE)
#### The features used are:
- SOS, EOS, lowercase, uppercase, title, digit, postag, previous_tag, next_tag
#### Feel free to include more features

In [187]:
def token_to_features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

### Running the feature extractor on the training data 
- Feature extraction
- Train-test-split

In [188]:
def find_features(sentence):
    return [token_to_features(sentence, i) for i in range(len(sentence))]

def find_labels(sentence):
    return [word[2] for word in sentence]
    

# Extract features for training and testing data
features = [find_features(sentence) for sentence in sample_data]
labels = [find_labels(sentence) for sentence in sample_data]

In [189]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=0)

### Training the CRF model with the features extracted using the feature extractor method

In [190]:
# Set the parameters of the model
crf = CRF(algorithm='lbfgs', c1=0.1, c2=10, max_iterations=50)
crf.fit(X_train, y_train)

# Providing a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
joblib.dump(crf, 'crf_model.joblib')

['crf_model.joblib']

### Predicting the test data with the built model

In [191]:
labels = list(crf.classes_)
sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))
y_pred = crf.predict(X_test)
y_pred

[['Method', 'Qty', 'Form', 'PeriodUnit'],
 ['PeriodUnit'],
 ['Method', 'Qty', 'Form', 'Period', 'PeriodUnit'],
 ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'],
 ['Method', 'Qty', 'Form', 'Qty', 'Form'],
 ['Method', 'Qty', 'PeriodUnit'],
 ['Period', 'PeriodUnit'],
 ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'],
 ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'],
 ['Qty', 'Form', 'Duration', 'DurationUnit', 'PeriodUnit'],
 ['EVERY', 'PeriodUnit'],
 ['Method', 'Qty', 'Form', 'Qty', 'Form']]

### Compute Score

In [192]:
print(f"F-Score = {metrics.flat_f1_score(y_test, y_pred,average='weighted', labels=labels)}")

F-Score = 0.7093627412776349


m:\GBC\ML2\.ml2-venv\lib\site-packages\sklearn\metrics\_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


### Compute Classification Report

In [193]:
print(metrics.flat_classification_report(y_test, y_pred, labels=sorted_labels, digits=3))

              precision    recall  f1-score   support

           M      0.000     0.000     0.000         0
       Q4-6H      0.000     0.000     0.000         0
        Q46H      0.000     0.000     0.000         0
       DAILY      0.000     0.000     0.000         1
      BEFORE      0.000     0.000     0.000         0
       AFTER      0.000     0.000     0.000         0
        THEN      0.000     0.000     0.000         0
        WHEN      1.000     1.000     1.000         1
         BID      0.000     0.000     0.000         1
         QID      0.000     0.000     0.000         1
         TID      0.000     0.000     0.000         1
       TIMES      0.000     0.000     0.000         0
        WITH      0.000     0.000     0.000         0
         AND      0.000     0.000     0.000         0
          PO      0.000     0.000     0.000         0
          TO      0.000     0.000     0.000         2
        FOOD      0.000     0.000     0.000         0
         FOR      0.000    

m:\GBC\ML2\.ml2-venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
m:\GBC\ML2\.ml2-venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
m:\GBC\ML2\.ml2-venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
m:\GBC\ML2\.ml2-venv\lib\site-packages\sklearn\metrics\_classification.py:1

### Plot Confusion Matrix

In [194]:
def print_confusion_matrix(cm, labels):
    print("\n")
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        sum = 0
        for j in range(len(labels)):
            cell = "%{0}.0f".format(columnwidth) % cm[i, j]
            sum =  sum + int(cell)
            print(cell, end=" ")
        print(sum) #Prints the total number of instances per cat at the end.

In [195]:
def get_confusion_matrix(y_true,y_pred,labels):
    trues,preds = [], []
    for yseq_true, yseq_pred in zip(y_true, y_pred):
        trues.extend(yseq_true)
        preds.extend(yseq_pred)
    print_confusion_matrix(confusion_matrix(trues,preds,labels=labels),labels)

In [196]:
get_confusion_matrix(y_test, y_pred,labels=sorted_labels)



                            M        Q4-6H         Q46H        DAILY       BEFORE        AFTER         THEN         WHEN          BID          QID          TID        TIMES         WITH          AND           PO           TO         FOOD          FOR           AT        EVERY           BY       Period    PeriodMax   PeriodUnit       Method         Form    Frequency          Qty     Duration  DurationMax DurationUnit 
               M            0            0            0            0            0            0            0            0            0            0            0            0            0            0            0            0            0            0            0            0            0            0            0            0            0            0            0            0            0            0            0 0
           Q4-6H            0            0            0            0            0            0            0            0            0            0         

### Putting all the prediction logic inside a predict method

In [197]:
def predict(sig):
    """
    predict(sig)
    Purpose: Labels the given sig into corresponding labels
    @param sig. A Sentence  # A medical prescription sig written by a doctor
    @return     A list      # A list with predicted labels (first level of labeling)
    >>> predict('2 tabs every 4 hours')
    [['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]
    >>> predict('2 tabs with food')
    [['Qty', 'Form', 'WITH', 'FOOD']]
    >>> predict('2 tabs qid x 30 days')
    [['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]
    """
    doc = pos_tag(sig.split())
    feature = [token_to_features(doc, i) for i in range(len(doc))]
    predictions = crf.predict([feature])
    print(sig)
    print(predictions)
    return predictions

### Sample predictions

In [198]:
predictions = predict("take 2 tabs every 6 hours x 10 days")

take 2 tabs every 6 hours x 10 days
[['Method', 'Qty', 'Form', 'EVERY', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [199]:
predictions = predict("2 capsu for 10 day at bed")

2 capsu for 10 day at bed
[['Qty', 'Form', 'FOR', 'Duration', 'Form', 'Period', 'PeriodUnit']]


In [200]:
predictions = predict("2 capsu for 10 days at bed")

2 capsu for 10 days at bed
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Period', 'PeriodUnit']]


In [201]:
predictions = predict("5 days 2 tabs at bed")

5 days 2 tabs at bed
[['Qty', 'Form', 'Qty', 'Form', 'BEFORE', 'WHEN']]


In [202]:
predictions = predict("3 tabs qid x 10 weeks")

3 tabs qid x 10 weeks
[['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit']]


In [203]:
predictions = predict("x 30 days")

x 30 days
[['Method', 'Qty', 'DurationUnit']]


In [204]:
predictions = predict("x 20 months")

x 20 months
[['Method', 'Qty', 'Form']]


In [205]:
predictions = predict("take 2 tabs po tid for 10 days")

take 2 tabs po tid for 10 days
[['Method', 'Qty', 'Form', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [206]:
predictions = predict("take 2 capsules po every 6 hours")

take 2 capsules po every 6 hours
[['Method', 'Qty', 'Form', 'Frequency', 'EVERY', 'Period', 'PeriodUnit']]


In [207]:
predictions = predict("inject 2 units pu tid")

inject 2 units pu tid
[['Method', 'Qty', 'Form', 'Period', 'PeriodUnit']]


In [208]:
predictions = predict("swallow 3 caps tid by mouth")

swallow 3 caps tid by mouth
[['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit']]


In [209]:
predictions = predict("inject 3 units orally")

inject 3 units orally
[['Method', 'Qty', 'Form', 'PeriodUnit']]


In [210]:
predictions = predict("orally take 3 tabs tid")

orally take 3 tabs tid
[['Method', 'Method', 'Qty', 'Form', 'PeriodUnit']]


In [211]:
predictions = predict("by mouth take three caps")

by mouth take three caps
[['Qty', 'Form', 'Method', 'Qty', 'Form']]


In [212]:
predictions = predict("take 3 tabs orally three times a day for 10 days at bedtime")

take 3 tabs orally three times a day for 10 days at bedtime
[['Method', 'Qty', 'Form', 'Method', 'Qty', 'Form', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit', 'BEFORE', 'WHEN']]


In [213]:
predictions = predict("take 3 tabs orally bid for 10 days at bedtime")

take 3 tabs orally bid for 10 days at bedtime
[['Method', 'Qty', 'Form', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'BEFORE', 'WHEN']]


In [214]:
predictions = predict("take 3 tabs bid orally at bed")

take 3 tabs bid orally at bed
[['Method', 'Qty', 'Form', 'Qty', 'Form', 'Period', 'PeriodUnit']]


In [215]:
predictions = predict("take 10 capsules by mouth qid")

take 10 capsules by mouth qid
[['Method', 'Qty', 'Form', 'Period', 'Period', 'PeriodUnit']]


In [216]:
predictions = predict("inject 10 units orally qid x 3 months")

inject 10 units orally qid x 3 months
[['Method', 'Qty', 'Form', 'Qty', 'Form', 'Method', 'Qty', 'Form']]


In [217]:
prediction = predict("please take 2 tablets per day for a month in the morning and evening each day")

please take 2 tablets per day for a month in the morning and evening each day
[['Method', 'Method', 'Qty', 'Form', 'Period', 'PeriodUnit', 'FOR', 'Period', 'PeriodUnit', 'Period', 'Period', 'PeriodUnit', 'Qty', 'Form', 'Period', 'PeriodUnit']]


In [218]:
prediction = predict("Amoxcicillin QID 30 tablets")

Amoxcicillin QID 30 tablets
[['Method', 'Method', 'Qty', 'Form']]


In [219]:
prediction = predict("take 3 tabs TID for 90 days with food")

take 3 tabs TID for 90 days with food
[['Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'Period', 'PeriodUnit']]


In [220]:
prediction = predict("with food take 3 tablets per day for 90 days")

with food take 3 tablets per day for 90 days
[['Qty', 'Form', 'Method', 'Qty', 'Form', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [221]:
prediction = predict("with food take 3 tablets per week for 90 weeks")

with food take 3 tablets per week for 90 weeks
[['Qty', 'Form', 'Method', 'Qty', 'Form', 'Qty', 'Form', 'FOR', 'Duration', 'PeriodUnit']]


In [222]:
prediction = predict("take 2-4 tabs")

take 2-4 tabs
[['Method', 'Qty', 'Form']]


In [223]:
prediction = predict("take 2 to 4 tabs")

take 2 to 4 tabs
[['Method', 'Qty', 'Form', 'Qty', 'Form']]


In [224]:
prediction = predict("take two to four tabs")

take two to four tabs
[['Method', 'Qty', 'Form', 'Qty', 'Form']]


In [225]:
prediction = predict("take 2-4 tabs for 8 to 9 days")

take 2-4 tabs for 8 to 9 days
[['Method', 'Qty', 'Form', 'FOR', 'Qty', 'Form', 'Qty', 'DurationUnit']]


In [226]:
prediction = predict("take 20 tabs every 6 to 8 days")

take 20 tabs every 6 to 8 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit', 'Qty', 'DurationUnit']]


In [227]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Qty', 'DurationUnit']]


In [228]:
prediction = predict("take 2 tabs every 2 to 10 weeks")

take 2 tabs every 2 to 10 weeks
[['Method', 'Qty', 'Form', 'EVERY', 'Qty', 'Form', 'Qty', 'PeriodUnit']]


In [229]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Qty', 'DurationUnit']]


In [230]:
prediction = predict("take 2 tabs every 2 to 10 months")

take 2 tabs every 2 to 10 months
[['Method', 'Qty', 'Form', 'EVERY', 'Qty', 'Form', 'Qty', 'Form']]


In [231]:
prediction = predict("every 60 mins")

every 60 mins
[['EVERY', 'Period', 'PeriodUnit']]


In [232]:
prediction = predict("every 10 mins")

every 10 mins
[['EVERY', 'Period', 'PeriodUnit']]


In [233]:
prediction = predict("every two to four months")

every two to four months
[['EVERY', 'Period', 'TO', 'Period', 'PeriodUnit']]


In [234]:
prediction = predict("take 2 tabs every 3 to 4 days")

take 2 tabs every 3 to 4 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Qty', 'DurationUnit']]


In [235]:
prediction = predict("every 3 to 4 days take 20 tabs")

every 3 to 4 days take 20 tabs
[['EVERY', 'Period', 'TO', 'Qty', 'Form', 'Method', 'Qty', 'Form']]


In [236]:
prediction = predict("once in every 3 days take 3 tabs")

once in every 3 days take 3 tabs
[['Qty', 'Form', 'EVERY', 'Qty', 'Form', 'Method', 'Qty', 'Form']]


In [237]:
prediction = predict("take 3 tabs once in every 3 days")

take 3 tabs once in every 3 days
[['Method', 'Qty', 'Form', 'Frequency', 'Period', 'EVERY', 'Period', 'DurationUnit']]


In [238]:
prediction = predict("orally take 20 tabs every 4-6 weeks")

orally take 20 tabs every 4-6 weeks
[['Method', 'Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]


In [239]:
prediction = predict("10 tabs x 2 days")

10 tabs x 2 days
[['Qty', 'Form', 'Method', 'Qty', 'DurationUnit']]


In [240]:
prediction = predict("3 capsule x 15 days")

3 capsule x 15 days
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [241]:
prediction = predict("10 tabs")

10 tabs
[['Qty', 'Form']]
